In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# #定义最大灰度级数
# gray_level = 128

# def maxGrayLevel(img):
#     max_gray_level=0
#     (height,width)=img.shape
#     for y in range(height):
#         for x in range(width):
#             if img[y][x] > max_gray_level:
#                 max_gray_level = img[y][x]
#     return max_gray_level+1

# def getGlcm(input,d_x,d_y):
#     srcdata=input.copy()
#     ret=[[0.0 for i in range(gray_level)] for j in range(gray_level)]
#     (height,width) = input.shape

#     max_gray_level=maxGrayLevel(input)

#     #若灰度级数大于gray_level，则将图像的灰度级缩小至gray_level，减小灰度共生矩阵的大小
#     if max_gray_level > gray_level:
#         for j in range(height):
#             for i in range(width):
#                 srcdata[j][i] = srcdata[j][i]*gray_level / max_gray_level

#     for j in range(height-d_y):
#         for i in range(width-d_x):
#             rows = srcdata[j][i]
#             cols = srcdata[j + d_y][i+d_x]
#             ret[rows][cols]+=1.0

#     for i in range(gray_level):
#         for j in range(gray_level):
#             ret[i][j]/=float(height*width)

#     return np.array(ret)
    
# def batch_GLCM(images):
#     greycomatrix_list = []
#     for i in tqdm(range(len(images))):
#         img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
#         gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         glcm_list = []
        
#         glcm_0=getGlcm(gray, 1,0)
#         glcm_0=cv2.normalize(glcm_0, glcm_0, 0, 1, cv2.NORM_MINMAX) 
#         glcm_list.append(glcm_0)
        
#         glcm_1=getGlcm(gray, 0,1)
#         glcm_1=cv2.normalize(glcm_1, glcm_1, 0, 1, cv2.NORM_MINMAX) 
#         glcm_list.append(glcm_1)
        
#         glcm_2=getGlcm(gray, 1,1)
#         glcm_2=cv2.normalize(glcm_2, glcm_2, 0, 1, cv2.NORM_MINMAX) 
#         glcm_list.append(glcm_2)

#         glcm_list=np.array(glcm_list,dtype = float32)
#         greycomatrix_list.append(glcm_list)
        
#     greycomatrix_list = np.array(greycomatrix_list,dtype = float32).reshape([images.shape[0],images.shape[1],images.shape[2],3)
    
#     print (greycomatrix_list.shape)
#     return greycomatrix_list

In [3]:
# def atoi(text):
#     return int(text) if text.isdigit() else text

# def natural_keys(text):
#     return [ atoi(c) for c in re.split('(\d+)', text) ]

# def load_flower_data():
#     # grab the list of images that we'll be describing
#     print("[INFO] handling images...")
#     TRAIN_ORIGINAL_DIR = '../train/'
#     TRAIN_SUB_DIR = '../subsample/'
#     TRAIN_GAN = '../../image_gan/'
#     TEST_DIR = '../../test/'

#     # use this for full dataset
#     train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
#     test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
#     train_images = train_images_gan
    
#     train_images.sort(key=natural_keys)
#     test_images.sort(key=natural_keys)

#     # initialize the features matrix and labels list
#     trainImage = []
#     trainLabels = []
#     testImage = []
#     testLabels = []

#     # loop over the input images
#     for (i, imagePath) in enumerate(train_images):
#         # extract the class label
#         # get the labels from the name of the images by extract the string before "_"
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # read and resize image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         trainImage.append(img)
#         trainLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
#       # loop over the input images
#     for (i, imagePath) in enumerate(test_images):
#         # extract the class label
#         # our images were named as labels.image_number.format
#         # get the labels from the name of the images by extract the string before "."
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # extract CNN features in the image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         testImage.append(img)
#         testLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


#     trainImage = np.array(trainImage,dtype = float32)
#     trainLabels = np.array(trainLabels)
#     testImage = np.array(testImage,dtype = float32)
#     testLabels = np.array(testLabels)
#     print (trainImage.shape)
    
#     trainImage = trainImage.astype(np.float32) / 255
#     testImage = testImage.astype(np.float32) / 255
    
#     le = preprocessing.LabelEncoder()
#     le.fit(trainLabels)
#     list(le.classes_)
#     trainLabels = le.transform(trainLabels) 
#     testLabels = le.transform(testLabels) 
    
#     return trainImage, trainLabels, testImage, testLabels

In [4]:
# trainImage, trainLabels, testImage, testLabels = load_flower_data()

# trainImage_GLCM = batch_GLCM(trainImage)
# testImage_GLCM  = batch_GLCM(testImage)
# nb_classes = 2

# # Convert class vectors to binary class matrices.
# trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
# print (trainLabels)
# testLabels = keras.utils.to_categorical(testLabels, nb_classes)
# print (testLabels)
# print (testLabels.shape)

# np.save('../trainImage.npy', trainImage)
# np.save('../trainLabels.npy', trainLabels)
# np.save('../testImage.npy', testImage)
# np.save('../testLabels.npy', testLabels)
# np.save('../trainImage_GLCM.npy', trainImage_GLCM)
# np.save('../testImage_GLCM.npy', testImage_GLCM)

# print("[INFO] trainImage matrix: {:.2f}MB".format(
#     (trainImage.nbytes) / (1024 * 1000.0)))
# print("[INFO] trainLabels matrix: {:.4f}MB".format(
#     (trainLabels.nbytes) / (1024 * 1000.0)))
# print("[INFO] testImage matrix: {:.2f}MB".format(
#     (testImage.nbytes) / (1024 * 1000.0)))
# print("[INFO] testLabels matrix: {:.4f}MB".format(
#     (testLabels.nbytes) / (1024 * 1000.0)))
# print("[INFO] trainImage_GLCM matrix: {:.2f}MB".format(
#     (trainImage_GLCM.nbytes) / (1024 * 1000.0)))
# print("[INFO] testImage_GLCM matrix: {:.4f}MB".format(
#     (testImage_GLCM.nbytes) / (1024 * 1000.0)))


In [5]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_GLCM_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_glcm = np.load('../trainImage_GLCM.npy')
        self.test_x_glcm = np.load('../testImage_GLCM.npy')
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_GLCM, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_GLCM",x_GLCM.get_shape()) # 128, 128, 3 
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_GLCM,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_GLCM = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_GLCM')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_GLCM, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.x_GLCM, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_glcm = self.train_x_glcm[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
#                 batch_x_GLCM = self.batch_GLCM(batch_x)
                batch_x_GLCM = shuffled_set_glcm[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_GLCM: batch_x_GLCM,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_GLCM_test =self.test_x_glcm[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_GLCM: batch_x_GLCM_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_GLCM_test =self.test_x_glcm[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_GLCM: batch_x_GLCM_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_GLCM_test =self.test_x_glcm[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_GLCM: batch_x_GLCM_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [6]:
dataset = '4_Flowers_1s'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
# [*] Best Epoch:  6 , Accuracy:  0.8399999737739563
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_GLCM_C5_D1_Kernel(3,3)_128_lrdecay/CNN_GLCM_C5_D1_Kernel(3,3)_128_lrdecay-6
#  [*] Finished testing Best Epoch: 6 , accuracy:  0.8400000333786011 !

#  [*] Training finished!
#  [*] Best Epoch:  4 , Accuracy:  0.800000011920929
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_GLCM_C5_D1_Kernel(3,3)_128_lrdecay/CNN_GLCM_C5_D1_Kernel(3,3)_128_lrdecay-4
#  [*] Finished testing Best Epoch: 4 , accuracy:  0.7999999523162842 !

CNN:x (100, 128, 128, 3)
CNN:x_GLCM (100, 128, 128, 3)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_GLCM (100, 128, 128, 3)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_1_2/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_2

Epoch: [ 5] [  16/  20] time: 27.4746, loss: 0.04339203
Epoch: [ 5] [  17/  20] time: 27.7169, loss: 0.01382364
Epoch: [ 5] [  18/  20] time: 27.9605, loss: 0.07291513
Epoch: [ 5] [  19/  20] time: 28.2012, loss: 0.02691477
[5/50] - ptime: 5.1461 loss: 0.03660614 acc: 0.73000 lr: 0.00100000
Epoch: [ 6] [   0/  20] time: 29.1237, loss: 0.03946025
Epoch: [ 6] [   1/  20] time: 29.3623, loss: 0.00837838
Epoch: [ 6] [   2/  20] time: 29.6025, loss: 0.03398918
Epoch: [ 6] [   3/  20] time: 29.8443, loss: 0.00447670
Epoch: [ 6] [   4/  20] time: 30.0850, loss: 0.01058676
Epoch: [ 6] [   5/  20] time: 30.3250, loss: 0.02505489
Epoch: [ 6] [   6/  20] time: 30.5653, loss: 0.00188113
Epoch: [ 6] [   7/  20] time: 30.8067, loss: 0.02658002
Epoch: [ 6] [   8/  20] time: 31.0481, loss: 0.01845208
Epoch: [ 6] [   9/  20] time: 31.2884, loss: 0.03521707
Epoch: [ 6] [  10/  20] time: 31.5289, loss: 0.01270238
Epoch: [ 6] [  11/  20] time: 31.7706, loss: 0.02620232
Epoch: [ 6] [  12/  20] time: 32.011

Epoch: [12] [  14/  20] time: 65.5422, loss: 0.02104071
Epoch: [12] [  15/  20] time: 65.7847, loss: 0.02581180
Epoch: [12] [  16/  20] time: 66.0267, loss: 0.00380945
Epoch: [12] [  17/  20] time: 66.2683, loss: 0.01164910
Epoch: [12] [  18/  20] time: 66.5088, loss: 0.03605047
Epoch: [12] [  19/  20] time: 66.7517, loss: 0.02180663
[12/50] - ptime: 5.1489 loss: 0.01555224 acc: 0.43000 lr: 0.00090000
Epoch: [13] [   0/  20] time: 67.6481, loss: 0.03406337
Epoch: [13] [   1/  20] time: 67.8897, loss: 0.02058636
Epoch: [13] [   2/  20] time: 68.1305, loss: 0.00695443
Epoch: [13] [   3/  20] time: 68.3712, loss: 0.00817413
Epoch: [13] [   4/  20] time: 68.6124, loss: 0.01947048
Epoch: [13] [   5/  20] time: 68.8552, loss: 0.05369007
Epoch: [13] [   6/  20] time: 69.0977, loss: 0.00159705
Epoch: [13] [   7/  20] time: 69.3394, loss: 0.00715176
Epoch: [13] [   8/  20] time: 69.5813, loss: 0.03193051
Epoch: [13] [   9/  20] time: 69.8243, loss: 0.02742330
Epoch: [13] [  10/  20] time: 70.06

Epoch: [19] [  12/  20] time: 103.6238, loss: 0.01698836
Epoch: [19] [  13/  20] time: 103.8666, loss: 0.01064921
Epoch: [19] [  14/  20] time: 104.1088, loss: 0.01718762
Epoch: [19] [  15/  20] time: 104.3494, loss: 0.00958062
Epoch: [19] [  16/  20] time: 104.5928, loss: 0.00865076
Epoch: [19] [  17/  20] time: 104.8409, loss: 0.00556731
Epoch: [19] [  18/  20] time: 105.0844, loss: 0.01060153
Epoch: [19] [  19/  20] time: 105.3263, loss: 0.01180122
[19/50] - ptime: 5.1850 loss: 0.00774854 acc: 0.69000 lr: 0.00090000
Epoch: [20] [   0/  20] time: 106.2264, loss: 0.00872159
Epoch: [20] [   1/  20] time: 106.4676, loss: 0.00530761
Epoch: [20] [   2/  20] time: 106.7095, loss: 0.00512472
Epoch: [20] [   3/  20] time: 106.9520, loss: 0.01703656
Epoch: [20] [   4/  20] time: 107.1939, loss: 0.01217368
Epoch: [20] [   5/  20] time: 107.4365, loss: 0.00643317
Epoch: [20] [   6/  20] time: 107.6773, loss: 0.00235050
Epoch: [20] [   7/  20] time: 107.9189, loss: 0.00014049
Epoch: [20] [   8/ 

Epoch: [26] [   8/  20] time: 141.2073, loss: 0.00405416
Epoch: [26] [   9/  20] time: 141.4480, loss: 0.03468641
Epoch: [26] [  10/  20] time: 141.6890, loss: 0.01032505
Epoch: [26] [  11/  20] time: 141.9302, loss: 0.00619538
Epoch: [26] [  12/  20] time: 142.1717, loss: 0.00087243
Epoch: [26] [  13/  20] time: 142.4138, loss: 0.01110187
Epoch: [26] [  14/  20] time: 142.6552, loss: 0.01058914
Epoch: [26] [  15/  20] time: 142.8981, loss: 0.07469320
Epoch: [26] [  16/  20] time: 143.1416, loss: 0.02596756
Epoch: [26] [  17/  20] time: 143.3843, loss: 0.00183906
Epoch: [26] [  18/  20] time: 143.6257, loss: 0.00742525
Epoch: [26] [  19/  20] time: 143.8697, loss: 0.02007992
[26/50] - ptime: 5.1581 loss: 0.01162188 acc: 0.79000 lr: 0.00081000
Epoch: [27] [   0/  20] time: 144.7768, loss: 0.01136899
Epoch: [27] [   1/  20] time: 145.0170, loss: 0.01174152
Epoch: [27] [   2/  20] time: 145.2576, loss: 0.00396871
Epoch: [27] [   3/  20] time: 145.4992, loss: 0.00771452
Epoch: [27] [   4/ 

Epoch: [33] [   4/  20] time: 178.9445, loss: 0.01300298
Epoch: [33] [   5/  20] time: 179.1878, loss: 0.00255787
Epoch: [33] [   6/  20] time: 179.4291, loss: 0.00457131
Epoch: [33] [   7/  20] time: 179.6720, loss: 0.00005373
Epoch: [33] [   8/  20] time: 179.9159, loss: 0.00121061
Epoch: [33] [   9/  20] time: 180.1588, loss: 0.00709857
Epoch: [33] [  10/  20] time: 180.4009, loss: 0.00480068
Epoch: [33] [  11/  20] time: 180.6437, loss: 0.00980325
Epoch: [33] [  12/  20] time: 180.8869, loss: 0.00164832
Epoch: [33] [  13/  20] time: 181.1282, loss: 0.00685318
Epoch: [33] [  14/  20] time: 181.3694, loss: 0.00176657
Epoch: [33] [  15/  20] time: 181.6103, loss: 0.00129133
Epoch: [33] [  16/  20] time: 181.8529, loss: 0.00015370
Epoch: [33] [  17/  20] time: 182.0939, loss: 0.00129503
Epoch: [33] [  18/  20] time: 182.3359, loss: 0.00011580
Epoch: [33] [  19/  20] time: 182.5774, loss: 0.00063242
[33/50] - ptime: 5.1615 loss: 0.00363849 acc: 0.60000 lr: 0.00072900
Epoch: [34] [   0/ 

Epoch: [40] [   0/  20] time: 216.9644, loss: 0.00000253
Epoch: [40] [   1/  20] time: 217.2086, loss: 0.00007616
Epoch: [40] [   2/  20] time: 217.4506, loss: 0.00055342
Epoch: [40] [   3/  20] time: 217.6928, loss: 0.00048442
Epoch: [40] [   4/  20] time: 217.9351, loss: 0.00010700
Epoch: [40] [   5/  20] time: 218.1778, loss: 0.00000299
Epoch: [40] [   6/  20] time: 218.4222, loss: 0.00003661
Epoch: [40] [   7/  20] time: 218.6642, loss: 0.00002212
Epoch: [40] [   8/  20] time: 218.9082, loss: 0.00070025
Epoch: [40] [   9/  20] time: 219.1546, loss: 0.00177304
Epoch: [40] [  10/  20] time: 219.3967, loss: 0.00008652
Epoch: [40] [  11/  20] time: 219.6402, loss: 0.00012571
Epoch: [40] [  12/  20] time: 219.8833, loss: 0.00000245
Epoch: [40] [  13/  20] time: 220.1261, loss: 0.00002671
Epoch: [40] [  14/  20] time: 220.3687, loss: 0.00458602
Epoch: [40] [  15/  20] time: 220.6119, loss: 0.00399687
Epoch: [40] [  16/  20] time: 220.8567, loss: 0.00000008
Epoch: [40] [  17/  20] time: 2

Epoch: [46] [  17/  20] time: 254.1982, loss: 0.00009304
Epoch: [46] [  18/  20] time: 254.4393, loss: 0.00000051
Epoch: [46] [  19/  20] time: 254.6806, loss: 0.00006424
[46/50] - ptime: 5.1627 loss: 0.00015843 acc: 0.55000 lr: 0.00065610
Epoch: [47] [   0/  20] time: 255.5739, loss: 0.00000152
Epoch: [47] [   1/  20] time: 255.8159, loss: 0.00003849
Epoch: [47] [   2/  20] time: 256.0586, loss: 0.00031726
Epoch: [47] [   3/  20] time: 256.3058, loss: 0.00003464
Epoch: [47] [   4/  20] time: 256.5476, loss: 0.00074978
Epoch: [47] [   5/  20] time: 256.7903, loss: 0.00004019
Epoch: [47] [   6/  20] time: 257.0327, loss: 0.00000534
Epoch: [47] [   7/  20] time: 257.2775, loss: 0.00000691
Epoch: [47] [   8/  20] time: 257.5196, loss: 0.00000030
Epoch: [47] [   9/  20] time: 257.7656, loss: 0.00000194
Epoch: [47] [  10/  20] time: 258.0077, loss: 0.00001302
Epoch: [47] [  11/  20] time: 258.2506, loss: 0.00000024
Epoch: [47] [  12/  20] time: 258.4919, loss: 0.00019083
Epoch: [47] [  13/ 

In [7]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放